In [100]:
library(tidyverse)
library(feather)
library(lubridate)
library(magrittr)
library(stringr)

In [101]:
sh_en <- read_feather('../data/feather/en/sh_en.feather')
it_en <- read_feather('../data/feather/en/it_en.feather')
itc_en <- read_feather('../data/feather/en/itc_en.feather')
tr <- read_feather('../data/feather/tr.feather')

Warning message:
“Coercing int64 to double”Warning message:
“Coercing int64 to double”

In [102]:
item_age <- tr %>% 
            group_by(it_id) %>% 
            summarise(age = first(date, order_by = date),
                      recency = last(date, order_by = date),
                      growth = recency - age) 

it_en %<>% 
  left_join(item_age, by='it_id')

tr %<>% 
  left_join(it_en %>% select(it_id, age, recency, itc_id), by='it_id')

In [103]:
head(item_age)

it_id,age,recency,growth
0,2014-09-01,2014-09-01,0 days
1,2014-04-02,2014-10-20,201 days
2,2014-08-24,2014-11-12,80 days
3,2014-07-05,2014-08-26,52 days
4,2014-09-19,2014-09-19,0 days
5,2014-12-14,2014-12-14,0 days


In [104]:
# Reduce the item category to the below major categories
# seperate based on console type

itc_en %<>% 
  mutate(en_itc_name = str_to_lower(en_itc_name)) %>% 
  mutate(mtype = if_else(str_detect(en_itc_name, 'accessor'), 'acc', 
                  if_else(str_detect(en_itc_name, 'game consoles'), 'gc', 
                    if_else(str_detect(en_itc_name, 'games'), 'games',
                      if_else(str_detect(en_itc_name, 'books'), 'books',
                        if_else(str_detect(en_itc_name, 'programs'), 'programs',
                          if_else(str_detect(en_itc_name, 'gift'), 'gifts',
                            if_else(str_detect(en_itc_name, 'movie'), 'movie',
                              if_else(str_detect(en_itc_name, 'music'), 'music', 'misc')))))))))

itc_en %<>% 
  mutate(console_type = if_else(str_detect(en_itc_name, 'ps2'), 'ps2',
                          if_else(str_detect(en_itc_name, 'ps3'), 'ps3',
                             if_else(str_detect(en_itc_name, 'ps4'), 'ps4',
                               if_else(str_detect(en_itc_name, 'pc'), 'pc',
                                  if_else(str_detect(en_itc_name, 'ps4'), 'ps4',
                             if_else(str_detect(en_itc_name, 'xbox 360'), 'xbox 360',
                                     if_else(str_detect(en_itc_name, 'xbox one'), 'xbox one', 'zero'))))))))

In [105]:
tr %<>% 
  left_join(itc_en %>% select(itc_id, mtype), by='itc_id')

tr %<>% 
  mutate(week_age = as.numeric(difftime(date, age, units="weeks"))) %>% 
  select(-recency)
# Main data frame define
df <- tr %>% 
  group_by(dbn, s_id, it_id) %>% 
  summarise(sales = sum(count), avg_age = mean(week_age)) %>% 
  ungroup

In [106]:
head(df)

dbn,s_id,it_id,sales,avg_age
0,0,32,6,2.714286
0,0,33,3,2.142857
0,0,35,1,1.142857
0,0,43,1,3.571429
0,0,51,2,2.857143
0,0,61,1,0.000000


In [107]:
# Add previous month for joining with other df
df %<>% 
  mutate(prev_dbn = dbn-1)

# previous month sales average for a shop and total number 
# of items each shop selling
df %<>% 
  left_join(df %>% 
              group_by(dbn, s_id) %>% 
              summarize(shop_sales = mean(sales), it_count = n()),
            by=c("prev_dbn" = "dbn", "s_id"))

# previous month item average sales and total number of shop's selling them.
df %<>% 
left_join(df %>% 
            group_by(dbn, it_id) %>% 
            summarize(it_sales = mean(sales), shop_count = n()),
          by=c("prev_dbn" = "dbn", "it_id"))

# previous month shop, item sales
df %<>% 
  left_join(df %>% 
              select(dbn, s_id, it_id, sales) %>% 
              rename(prev_sales = sales),
            by = c("prev_dbn" = "dbn", "s_id", "it_id")) 


In [108]:
df %>% sample_n(20)

dbn,s_id,it_id,sales,avg_age,prev_dbn,shop_sales,it_count,it_sales,shop_count,prev_sales
18,56,2578,2,3.142857,17,1.787543,1445,2.727273,22,NA
6,47,14931,2,29.214286,5,1.841871,898,3.310345,29,2
4,51,18509,3,18.904762,3,1.651104,1178,1.285714,7,3
1,31,16229,5,5.642857,0,3.004430,3386,1.571429,7,1
20,31,11903,1,34.000000,19,2.943612,2802,1.375000,8,3
14,4,2854,4,62.678571,13,1.896350,685,2.096774,31,1
2,19,6340,1,8.428571,1,2.067104,1371,1.000000,1,NA
1,15,3077,1,7.714286,0,2.214634,1230,2.750000,28,3
32,42,21672,1,39.428571,31,2.448562,1565,1.100000,10,2
23,19,10748,2,1.142857,22,2.155242,992,NA,NA,NA


In [109]:
# Add lag features
lag_times = c(2, 3 ,6, 9, 12)

for (i in lag_times){
    lag_field  <- paste('lag_sales_', as.character(i), sep = '')
    
    df %<>% 
        mutate(lag_dbn = dbn-i) %>% 
        left_join(df %>% 
                    select(dbn, s_id, it_id, sales) %>% 
                    rename(!!lag_field := sales),
                  by = c("lag_dbn" = "dbn", "s_id", "it_id"))
}

In [141]:
df %>% sample_n(10)

dbn,s_id,it_id,sales,avg_age,prev_dbn,shop_sales,it_count,it_sales,shop_count,prev_sales,lag_dbn,lag_sales_2,lag_sales_3,lag_sales_6,lag_sales_9,lag_sales_12
28,31,21619,1,124.571429,27,3.060025,2399,1.250000,8,1,16,1,4,NA,4,1
30,47,13300,2,1.500000,29,2.148148,810,NA,NA,NA,18,NA,NA,NA,NA,NA
28,25,6625,2,56.357143,27,2.864146,2142,1.083333,12,1,16,1,1,1,NA,2
24,21,4334,1,9.857143,23,2.539448,1559,4.279070,43,2,12,1,NA,NA,NA,NA
11,31,7055,3,14.190476,10,3.504939,3341,1.733333,15,2,-1,5,14,NA,NA,NA
22,56,17890,2,24.285714,21,1.696123,1109,1.000000,5,NA,10,NA,2,2,NA,NA
20,28,7213,1,77.571429,19,3.263217,1778,1.724138,29,2,8,2,3,4,6,1
27,48,7895,2,119.142857,26,1.544855,758,2.047619,21,1,15,NA,2,1,1,NA
7,6,6635,1,32.285714,6,1.906924,1762,1.214286,14,NA,-5,NA,NA,NA,NA,NA
29,59,15063,1,109.428571,28,1.810277,506,1.428571,21,1,17,NA,2,2,NA,1


In [125]:
# Replace all na with 0

sales_columns <- str_detect(colnames(df), 'sales')

In [132]:
df[,sales_columns] %>% replace_na(0)

ERROR: Error: is_list(replace) is not TRUE


In [137]:
df[,sales_columns]  %>% replace_na(rep(0, nrow(.))) %>% head

ERROR: Error: is_list(replace) is not TRUE


In [139]:
seq(0, nrow(df))